# <center>本地部署开源大模型

## <center>Ch.11 大模型开发工具Transformers库的使用及配置大模型开发环境

&emsp;&emsp;本节课的内容重点，是和大家一起探讨一下大模型开发库 Hugging Face Transformers库的使用以及大模型开发环境的搭建。

&emsp;&emsp;截止目前，对于Qwen开源系列模型，我们已经掌握双系统下的私有化部署方法，而每个参数量下的大模型其对应的GitHub项目文件，为用户提供的一些快速启动该模型的代码逻辑和脚本，也能够顺利的运行。比如之前介绍的直接运行`web_demo.py`就可以启动Web UI的交互页面，直接运行`openai_api.py`就可以启动一个服务接口，支持远程调用。

&emsp;&emsp;但透过表面看本质，官方提供的Demo级别的代码，对于私有化大模型做上层应用开发的需求来说，大家应该也能感受到是肯定是无法直接应用的。私有化业务，必然要涉及到更高级的代码封装，比如异步、高性能计算，数据的处理等高阶优化操作，特别是：目前有非常多的推理加速开源框架，其一是能节省显存，其二又能加速推理效率，那么如果去将自己使用的开源大模型集成到某个框架中，并且基于此做更高级的抽象，也必然是实际工作中亟需的工作。所以如何去理解大模型的启动过程，参与推理和训练每个阶段的关键点，我们需要有一个比较明确的认识。

&emsp;&emsp;本节内容，我们就来一点一点的撕破大模型推理的背后原理，通过Transformers库和具体的代码，来理解从用户输入Prompt到Qwen模型中，最终返回模型回复，都需要经过哪些主要的过程，最后手动实现Qwen-Chat类模型对vLLM开源框架的接入。

&emsp;&emsp;首先，当我们尝试一个新的开源大模型的时候，往往看到的都是这样的官方说明：

- **ChatGLM3-6B：https://github.com/THUDM/ChatGLM3**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311022764.png" width=100%></div>

- **Qwen-7B：https://github.com/QwenLM/Qwen/blob/main/README_CN.md**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311024443.png" width=100%></div>

&emsp;&emsp;可以看到，根据官方的描述，仅仅需要几行代码就可以建立起与某个开源大模型的对话交互。我们可以以Qwen-7B-Chat模型为例，随便打开一个可以运行Python的IDE来尝试运行。我这里使用Jupyter Lab：

&emsp;&emsp;看似简单的几行代码，却往往难住很多人无法成功运行的原因，主要是以下两点：

- **问题1：没有安装Pytorch** 

&emsp;&emsp;关于如何安装Pytorch，我们在本地部署大模型的每一节内容中都有详细的介绍，此处不再重复说明。

- **问题2：No module named 'transformers** 

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# 可选的模型包括: "Qwen/Qwen-7B-Chat", "Qwen/Qwen-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()

model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

# 第一轮对话
response, history = model.chat(tokenizer, "你好", history=None)
print(response)

ModuleNotFoundError: No module named 'transformers'

&emsp;&emsp;问题在于缺少transformers模块，也就是导致这两行代码不能执行：
```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
```

&emsp;&emsp;对应的解决办法是：使用Python的包管理工具pip,来安装transformers库。代码如下：

In [2]:
! pip install transformers==4.32.0

  Obtaining dependency information for transformers==4.32.0 from https://files.pythonhosted.org/packages/ae/95/283a1c004430bd2a9425d6937fc545dd49a4e4592feb76be0299a14e2378/transformers-4.32.0-py3-none-any.whl.metadata
     ---------------------------------------- 0.0/118.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/118.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/118.5 kB ? eta -:--:--
     ---------------------------------------- 0.0/118.5 kB ? eta -:--:--
     --- ------------------------------------ 10.2/118.5 kB ? eta -:--:--
     --------- --------------------------- 30.7/118.5 kB 330.3 kB/s eta 0:00:01
     ------------ ------------------------ 41.0/118.5 kB 281.8 kB/s eta 0:00:01
     ------------------- ----------------- 61.4/118.5 kB 328.2 kB/s eta 0:00:01
     ------------------------- ----------- 81.9/118.5 kB 383.3 kB/s eta 0:00:01
     ---------------------------- -------- 92.2/118.5 kB 350.1 kB/s eta 0:00:01
     -----------

&emsp;&emsp;然后再次尝试执行：

- **问题3：We couldn't connect to 'https://huggingface.co' to load this file ....** 

In [4]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# 可选的模型包括: "Qwen/Qwen-7B-Chat", "Qwen/Qwen-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()

model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

# 第一轮对话
response, history = model.chat(tokenizer, "你好", history=None)
print(response)

'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen-7B-Chat/resolve/main/tokenizer_config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000146AEFFBB50>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen-7B-Chat/resolve/main/tokenizer_config.json
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen-7B-Chat/resolve/main/tokenization_qwen.py (Caused by ConnectTimeoutError(<urllib3.connection.HTTPSConnection object at 0x00000146B55CFF10>, 'Connection to huggingface.co timed out. (connect timeout=10)'))' thrown while requesting HEAD https://huggingface.co/Qwen/Qwen-7B-Chat/resolve/main/tokenization_qwen.py
'HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /Qwen/Qwen-7B-Chat/resolve/main/config.json (Caused by ConnectTimeoutError(<urllib3.connection.HTT

OSError: We couldn't connect to 'https://huggingface.co' to load this file, couldn't find it in the cached files and it looks like Qwen/Qwen-7B-Chat is not the path to a directory containing a file named modeling_qwen.py.
Checkout your internet connection or see how to run the library in offline mode at 'https://huggingface.co/docs/transformers/installation#offline-mode'.

&emsp;&emsp;从报错信息上看，问题的原因在于，无法从'https://huggingface.co' 这个网站中，拉取到`Qwen/Qwen-7B-Chat`这个模型导致的无法运行，而 'https://huggingface.co' 这个网站，对于国内的网络是不通的。所以需要开启科学上网的环境，打开后是这样的一个网站：（在本机先开启科学上网后再进行尝试）

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311058143.png" width=100%></div>

&emsp;&emsp;解决的方式是：在本机环境下开启科学上网，然后再执行这段代码。前提是：需要额外在安装如下两个依赖包：

In [4]:
! pip install transformers_stream_generator einops

In [5]:
! pip install accelerate

  Obtaining dependency information for accelerate from https://files.pythonhosted.org/packages/a6/b9/44623bdb05595481107153182e7f4b9f2ef9d3b674938ad13842054dcbd8/accelerate-0.26.1-py3-none-any.whl.metadata
   ---------------------------------------- 0.0/270.9 kB ? eta -:--:--
   ---------------------------------------- 0.0/270.9 kB ? eta -:--:--
   - -------------------------------------- 10.2/270.9 kB ? eta -:--:--
   ---- ---------------------------------- 30.7/270.9 kB 435.7 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/270.9 kB 281.8 kB/s eta 0:00:01
   ----- --------------------------------- 41.0/270.9 kB 281.8 kB/s eta 0:00:01
   -------- ------------------------------ 61.4/270.9 kB 297.7 kB/s eta 0:00:01
   ------------- ------------------------- 92.2/270.9 kB 350.1 kB/s eta 0:00:01
   -------------------- ----------------- 143.4/270.9 kB 448.2 kB/s eta 0:00:01
   ------------------------ ------------- 174.1/270.9 kB 476.3 kB/s eta 0:00:01
   -----------------

&emsp;&emsp;**注意：安装完上述两个依赖包后，需要重启Jupyter Lab才会生效。**

In [1]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

# 可选的模型包括: "Qwen/Qwen-7B-Chat", "Qwen/Qwen-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()



model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

# 第一轮对话
response, history = model.chat(tokenizer, "你好", history=None)
print(response)

C:\Users\snowb\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\snowb\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

你好！有什么我能帮你的吗？


In [2]:
# 第一轮对话
response, history = model.chat(tokenizer, "请你详细的介绍一下你自己", history=None)
print(response)

我是来自阿里云的大规模语言模型，我叫通义千问。我的主要功能是帮助用户生成与给定词语相关的高质量文本，以满足各种应用场景的需求。

我可以根据不同的任务和场景，生成各种类型的文本，包括但不限于文章、故事、诗歌、新闻、报告、评论、指南、教程、代码等。无论你需要的是专业性强的内容，还是富有创意的文学作品，我都可以帮你轻松实现。

除此之外，我还具备多种能力，例如：回答问题、提供定义、解释和建议、将文本从一种语言翻译成另一种语言、总结文本、生成文本、写故事、分析情绪、提供建议、开发算法、编写代码等。

在使用过程中，我会不断学习和进步，不断提升自己的表现，以更好地为用户提供服务。如果您有任何问题或需要帮助，请随时告诉我，我会尽力为您提供支持。


&emsp;&emsp;如果顺利执行后，会是非常漫长的下载过程，这个过程下载的文件，就是我们在前期课程中本地部署Qwen模型中从 Hugging Face 或者 modelscope 社区下载到的模型权重。只不过，通过上述代码，会自动的从Hugging Face上下载到了本地的`C:\Users\Admin\.cache\huggingface\hub.` 这个文件中。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311123636.png" width=100%></div>

&emsp;&emsp;下载到本地后，下次启动默认就会从Cache中加载模型，不会重复下载。此时再执行代码就可以正常的完成输出，而如果出现如下错误，则说明本机的配置太低，不足以支撑Qwen-7B-Chat模型的运行条件，会直接导致当前的开发环境进程被强制终止：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311126074.png" width=100%></div>

&emsp;&emsp;解决办法只能是选择更高配置的机器来尝试。或者选择去加载更小的模型，比如Qwen的1.8B模型，但是官方也说明了，上述加载方式可选的模型仅包括: "Qwen/Qwen-7B-Chat" 和 "Qwen/Qwen-14B-Chat"，所以，如何去考虑自定义加载不同参数量的模型，也往往是很多人在无法解决硬件问题的迫切需求，同时这也是在实际的业务开发中，在大模型领域必须掌握的底层知识。

&emsp;&emsp;接下来，我们就来拆解一下为什么能通过上述几行代码，仅仅通过修改模型的名称，就可以加载到不同的大模型实现对话交互。

# 1. 大模型开发工具Transformers库

&emsp;&emsp;其实从我们上面在初次与大模型建立交互过程中可能出现的报错内容来看，起决定因素的在这样两行代码中：
```python
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig
```

&emsp;&emsp;这里导入的Transformers，是一个 Python库，可以允许用户下载和训练模型，最初是用于机器学习领域，而随着大模型的爆发，Hugging Face 也是迅速占领了大模型的市场，将功能扩展到包括多模态、计算机视觉和音频处理等其他用途的模型。

> Github地址：https://github.com/huggingface/transformers

> Hugging Face地址：https://huggingface.co/docs/transformers/index

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311139436.png" width=100%></div>

&emsp;&emsp;其价值在于：

- 🤗 Transformers 提供了数以千计的预训练模型，支持 100 多种语言的文本分类、信息抽取、问答、摘要、翻译、文本生成。它的宗旨是让最先进的 NLP 技术人人易用。

- 🤗 Transformers 提供了便于快速下载和使用的API，让你可以把预训练模型用在给定文本、在你的数据集上微调然后通过 model hub 与社区共享。同时，每个定义的 Python 模块均完全独立，方便修改和快速研究实验。

- 🤗 Transformers 支持三个最热门的深度学习库： Jax, PyTorch 以及 TensorFlow — 并与之无缝整合。你可以直接使用一个框架训练你的模型然后用另一个加载和推理。

&emsp;&emsp;简单理解，就是Transformers库，集成了大量的预训练模型，将其内部的调用方式已经做好了封装，所以才有我们看到的 模型加载、模型推理这种仅仅通过调用某个接口就可以实现。实际上调用大模型的推理是比较复杂的，只不过Transformers库在`暗处`给我们写好了而已，让我们仅仅关注如何使用，而不需要去了解复杂的原理和代码实现，从而降低模型的使用门槛。

&emsp;&emsp;Transformers集成了非常多的模型，在 Hugging Face：https://huggingface.co/docs/transformers/index 中可以找到详细的说明：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311156981.png" width=100%></div>

&emsp;&emsp;这里需要去关注的概念是：Pipeline。在Transformers中，每个任务都有一个关联的pipeline()，而为了进一步简化使用门槛，Transformers抽象出了一个通用的pipeline()，其中包含所有特定任务的pipelines。pipeline()会自动加载一个默认模型和一个能够进行任务推理的预处理类。它是这样的一个自动化过程：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311246034.png" width=100%></div>

&emsp;&emsp;这个过程的理解是：每个模型，根据其架构和训练数据，其输入数据和输出数据都是不一样的， Pre-Precessing过程，就是数据进入模型前的预处理，而Post-Precessing，是指输入数据经过模型的计算推理后，以哪种方式输出，比如分类模型，那其输出就是分类的标签， 翻译任务，就是将输入的文本翻译成指定的语言。

&emsp;&emsp;而Transformers所谓的通用pipeline()，就是集成不同的模型在同一套流程中，就像启动大模型一样，只需要修改模型，就可以了。接下来可以快速的测试一下：

&emsp;&emsp;pipeline API reference：https://huggingface.co/docs/transformers/v4.37.2/en/main_classes/pipelines

&emsp;&emsp;首先来尝试一下文本分类任务：

In [2]:
# 导入pipeline模块
from transformers import pipeline

In [3]:
# 实例化
text_task = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english and revision af0f99b (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english).
Using a pipeline without specifying a model name and revision in production is not recommended.


C:\Users\snowb\anaconda3\Lib\site-packages\huggingface_hub\file_download.py:137: UserWarning: `huggingface_hub` cache-system uses symlinks by default to efficiently store duplicated files but your machine does not support them in C:\Users\snowb\.cache\huggingface\hub. Caching files will still work but in a degraded version that might require more space on your disk. This warning can be disabled by setting the `HF_HUB_DISABLE_SYMLINKS_WARNING` environment variable. For more details, see https://huggingface.co/docs/huggingface_hub/how-to-cache#limitations.
To support symlinks on Windows, you either need to activate Developer Mode or to run Python as an administrator. In order to see activate developer mode, see this article: https://docs.microsoft.com/en-us/windows/apps/get-started/enable-your-device-for-development
  warnings.warn(message)


&emsp;&emsp;如果初次使用，会自动执行下载，下载路径就是：C:\Users\snowb\.cache\huggingface\hub。如果想自定义模型的默认下载路径，可以在执行代码前执行：
```python

import os

os.environ['HF_HOME'] =pathme/hf'
os.environ['HF_HUB_CACHE'] pathume/hf/hub'
```

&emsp;&emsp;待模型下载完成后，就可以直接利用pipeline实例调用：

In [4]:
text_task("今天的天气可真好啊")

[{'label': 'NEGATIVE', 'score': 0.7773503065109253}]

In [5]:
text_task("我特别喜欢你，你能嫁给我吗？")

[{'label': 'NEGATIVE', 'score': 0.9618343114852905}]

In [6]:
text_task("What a lovely day it is today")

[{'label': 'POSITIVE', 'score': 0.9998749494552612}]

&emsp;&emsp;再测试一个摘要任务：

In [8]:
Summarization_task = pipeline(task="summarization")

No model was supplied, defaulted to sshleifer/distilbart-cnn-12-6 and revision a4f8f3e (https://huggingface.co/sshleifer/distilbart-cnn-12-6).
Using a pipeline without specifying a model name and revision in production is not recommended.


In [11]:
Summarization_task("春节临近，各地年货市场也开始热闹起来。从批发市场到年货大集，人们采购的热情格外高涨。 \
                    位于浙江金华的“华东金华农产品物流中心”是全国重要的水果集散枢纽之一，销售范围覆盖浙江省内及周边江西、福建 \
                    、安徽等省市。根据市场统计，最近几天的日均交易量达1.02万吨，较前一周增长5.57%。")

[{'summary_text': ' 春节临近，各地年货市场也开始热闹起来 .   \xa0‘‘�’: ‘I’m sorry.’. ‘“”: ““\xa0”. “I”   I’ll be happy to see that.”’s a happy coincidence.   ‘A happy coincidence’? ‘It was a coincidence. It was an accident.'}]

&emsp;&emsp;其实能够直观的感受到，这种方式可以让我们非常快速的应用起某个领域的模型去生成特殊的任务。我们需要做的就是去选择Transformers库中已经支持的，且效果较好的模型，然后按照它的规范去生成结果。比如你想加载指定的模型：

In [12]:
# 通过model参数来指定具体的模型
# speech_recognizer = pipeline("automatic-speech-recognition", model="facebook/wav2vec2-base-960h")

&emsp;&emsp;Transformers之所以能集成如此多的模型使用同一套规范来提供服务，根本原因还是在于现在优秀的模型基本都是基于Transformer架构的，因为架构是一样的，所以并不会出现太过于大的差异。其整个Pipeline()的底层处理逻辑可以抽象如下：

In [13]:
text_task("This course is amazing")

[{'label': 'POSITIVE', 'score': 0.9998831748962402}]

&emsp;&emsp;如下图所示：Tokenizer部分，就是进入模型的Pre-Processing过程。我们的输入，无论是中文还是英文还是其他语言，对于模型来说，都是不认识的，所以需要Tokenizer这个过程来将自然语言做一系列的预处理工作，这包括：
1. 将输入的Prompt 按照某种规则切分成小的Tokens；
2. 每个小的Tokens，在该模型训练时所建立的词表，都有一个id来对应，也就是下图的input IDs
3. 添加特殊tokens、截断、填充等操作，以满足模型的输入要求。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311236687.png" width=100%></div>

> Tokenizer 参数参考：https://huggingface.co/docs/transformers/main/internal/tokenization_utils#transformers.tokenization_utils_base.PreTrainedTokenizerBase

&emsp;&emsp;对于上述的这样一种通用的Pipeline流程，Transformers 采用的是 `AutoClass` 统一接口的设计，抽象出`from_pretrained`方法来管理Tokenizer和Model，以便我们能够自动去检索到相关的模型，并读取其模型权重、配置文件和词表信息等。

In [7]:
from transformers import AutoTokenizer

# 可选的模型包括: "Qwen/Qwen-7B-Chat", "Qwen/Qwen-14B-Chat"
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

In [2]:
tokenizer("今天要下大雪了，过年回不去家怎么办？")

{'input_ids': [100644, 30534, 16872, 26288, 100167, 34187, 3837, 107954, 18397, 105580, 45629, 102572, 11319], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}

In [3]:
raw_inputs = ['今天天气不好。',
              "连续几天都会下大雪，过年回不去家怎么办？"]

In [6]:
print(tokenizer(raw_inputs))

{'input_ids': [[100644, 104307, 101132, 1773], [104005, 101437, 101938, 16872, 26288, 100167, 3837, 107954, 18397, 105580, 45629, 102572, 11319]], 'token_type_ids': [[0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], 'attention_mask': [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]}


In [7]:
output = tokenizer(raw_inputs)

# 直接遍历并打印输出结果
for key, value in output.items():
    print(f"{key}: {value}")

input_ids: [[100644, 104307, 101132, 1773], [104005, 101437, 101938, 16872, 26288, 100167, 3837, 107954, 18397, 105580, 45629, 102572, 11319]]
token_type_ids: [[0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask: [[1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


设置Padding。

In [21]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True, pad_token='<|endoftext|>', padding_side='left',)
output = tokenizer(raw_inputs,  padding=True)

# 直接遍历并打印输出结果
for key, value in output.items():
    print(f"{key}: {value}")

input_ids: [[151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 151643, 100644, 104307, 101132, 1773], [104005, 101437, 101938, 16872, 26288, 100167, 3837, 107954, 18397, 105580, 45629, 102572, 11319]]
token_type_ids: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]]
attention_mask: [[0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]]


In [24]:
# 假设已经有了tokenizer和raw_inputs
output = tokenizer(raw_inputs)

# 遍历每个输入字符串
for i, input_ids in enumerate(output['input_ids']):
    tokens = tokenizer.convert_ids_to_tokens(input_ids)
    print(f"输入 {i+1}:")
    
    # 创建一个字符串来存储格式化的输出
    formatted_output = ""
    for token, id in zip(tokens, input_ids):
        formatted_output += f"{token}({id}) "
    
    print(formatted_output + "\n")

输入 1:
b'\xe4\xbb\x8a\xe5\xa4\xa9'(100644) b'\xe5\xa4\xa9\xe6\xb0\x94'(104307) b'\xe4\xb8\x8d\xe5\xa5\xbd'(101132) b'\xe3\x80\x82'(1773) 

输入 2:
b'\xe8\xbf\x9e\xe7\xbb\xad'(104005) b'\xe5\x87\xa0\xe5\xa4\xa9'(101437) b'\xe9\x83\xbd\xe4\xbc\x9a'(101938) b'\xe4\xb8\x8b'(16872) b'\xe5\xa4\xa7'(26288) b'\xe9\x9b\xaa'(100167) b'\xef\xbc\x8c'(3837) b'\xe8\xbf\x87\xe5\xb9\xb4'(107954) b'\xe5\x9b\x9e'(18397) b'\xe4\xb8\x8d\xe5\x8e\xbb'(105580) b'\xe5\xae\xb6'(45629) b'\xe6\x80\x8e\xe4\xb9\x88\xe5\x8a\x9e'(102572) b'\xef\xbc\x9f'(11319) 



&emsp;&emsp;类似 b'\xe4\xbb\x8a\xe5\xa4\xa9' 这样的序列是字节串表示的 UTF-8 编码的汉字。在 Python 中，以 b 前缀开头的字面量表示字节串（byte strings）。这些字节串是由文本在 UTF-8 编码下转换成的字节序列。如果想转化回汉字，则输入如下代码：

In [25]:
token_bytes = b'\xe4\xbb\x8a\xe5\xa4\xa9'  # 汉字 "今天" 的 UTF-8 编码
token_str = token_bytes.decode('utf-8')  # 解码为字符串
print(token_str)  # 输出 "今天"

今天


&emsp;&emsp;经过了Tokenizer的处理，会将每个词的Input_ids映射到具体的Embedding向量，组成一个Embedding向量矩阵，输入到大模型中，执行内部的矩阵计算。可以简单的理解如下：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311439292.png" width=100%></div>

&emsp;&emsp;经过Tokenizer后得到词的向量矩阵后，接下来进入模型，对于大模型这种自回归模型来说，其生成文本的过程是这样的：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311443666.png" width=100%></div>

> Transformer架构的学习地址：https://jalammar.github.io/illustrated-transformer/

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311515513.png" width=100%></div>

&emsp;&emsp;Head，表示的是模型头，用来接在基础模型的后面，从而将hidden states文本表示进一步处理，用于具体的任务。

In [1]:
from transformers import AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained("Qwen/Qwen-7B-Chat", device_map="auto", trust_remote_code=True).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...


Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [2]:
model

QWenLMHeadModel(
  (transformer): QWenModel(
    (wte): Embedding(151936, 4096)
    (drop): Dropout(p=0.0, inplace=False)
    (rotary_emb): RotaryEmbedding()
    (h): ModuleList(
      (0-31): 32 x QWenBlock(
        (ln_1): RMSNorm()
        (attn): QWenAttention(
          (c_attn): Linear(in_features=4096, out_features=12288, bias=True)
          (c_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (attn_dropout): Dropout(p=0.0, inplace=False)
        )
        (ln_2): RMSNorm()
        (mlp): QWenMLP(
          (w1): Linear(in_features=4096, out_features=11008, bias=False)
          (w2): Linear(in_features=4096, out_features=11008, bias=False)
          (c_proj): Linear(in_features=11008, out_features=4096, bias=False)
        )
      )
    )
    (ln_f): RMSNorm()
  )
  (lm_head): Linear(in_features=4096, out_features=151936, bias=False)
)

&emsp;&emsp;可以看到，上述就是Qwen模型的神经网络信息。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401311236687.png" width=100%></div>

In [4]:
from transformers.generation import GenerationConfig
model.generation_config = GenerationConfig.from_pretrained("Qwen/Qwen-7B-Chat", trust_remote_code=True)

In [5]:
model.generation_config

GenerationConfig {
  "chat_format": "chatml",
  "do_sample": true,
  "eos_token_id": 151643,
  "max_new_tokens": 512,
  "max_window_size": 24000,
  "pad_token_id": 151643,
  "repetition_penalty": 1.1,
  "top_k": 0,
  "top_p": 0.8,
  "transformers_version": "4.32.1",
  "trust_remote_code": true
}

In [8]:
# 第一轮对话
response, history = model.chat(tokenizer, "你好", history=None)
print(response)

你好！有什么我能帮助你的吗？


&emsp;&emsp;这种利用Transformers库来加载和运行模型，是主流且通用的一种做法，理解其背后的核心逻辑后，就能明白不同模型再transformers库中的加载，主要涉及的也只是对输入数据的预处理（Pre-processing）和对输出数据的后处理（Post-processing）。对于官方提供的github项目文件，其实本质上无非也就是修改了Pre-precessing 和 Post-precessing的处理逻辑，可以将其封装进FastAPI框架来提供API服务，或者集成到Gradio框架以创建交互式的Web UI。而只要我们理解这个中间过程，就能够适配任何合适的框架，并进行高度定制化的程序开发。

# 2. 配置大模型开发环境（Jupyter Lab）

&emsp;&emsp;关于私有化部署的大模型，针对不同的开发场景，选择合适的工具至关重要。例如，进行对话测试、接口调用和数据清洗等任务时，Jupyter Lab 这种交互式环境非常适合。它允许我们直观地观察到每一步操作的输出，这对于数据分析和初步测试尤其有用。另一方面，当涉及到更复杂的软件开发项目时，例如执行模型微调、定制对话逻辑或使用像langchain这样的工具进行工程级应用开发，PyCharm 这种功能全面的集成开发环境（IDE）就显得更为合适。PyCharm 提供了代码调试、项目管理和版本控制等高级功能，这对于开发大型应用和维护复杂代码库非常重要。

&emsp;&emsp;因此，我们将分别介绍这两种工具的使用方法。大家可以根据具体需求灵活选择最合适的工具，以获得最佳的开发体验。

&emsp;&emsp;我们首先来看一下Jupyter Lab 如何加载本地部署或者远程服务器部署的大模型环境。

&emsp;&emsp;需要明确的是：如果想要在本地的集成开发环境（如 Jupyter Lab、PyCharm或VsCode等）中执行无论是在本地，还是远程服务器上部署的大模型项目的代码脚本或调用大模型服务时，必须首先进入创建大模型时所建立的虚拟环境。而这个过程，在任何开发工具（IDE）中，默认情况下并不会自动加载这个虚拟环境。因此，接下来我们将详细介绍如何在 Jupyter Lab 和 PyCharm 中手动加载指定的虚拟环境，以便在本地IDE中有效进行大型模型的相关开发工作。

## 2.1 本地部署大模型，本地Jupyter NoteBook加载

&emsp;&emsp;默认安装的Jupyter NoteBook/Lab在启动时，只能选择一个其默认的kernel环境，即：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740966.png" width=80%></div>

&emsp;&emsp;安装完Anaconda利用conda创建了虚拟环境，但是启动jupyter notebook之后却找不到虚拟环境，原因是**在虚拟环境下缺少kernel.json文件。**解决方法如下：

- **Step 1. 打开Conda的命令行终端**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261802275.png" width=80%></div>

- **Step 2. 安装ipykernel**

&emsp;&emsp;安装ipykernel的命令如下：
```bash
conda install ipykernel
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740968.png" width=80%></div>

- **Step 3. 找到想要使用的虚拟环境**

&emsp;&emsp;查询虚拟环境指令如下：
```bash
conda env list
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740969.png" width=80%></div>

- **Step 4. 创建虚拟环境的kernel文件**

&emsp;&emsp;找到项目文件的虚拟环境后，对其创建kernel文件，执行命令如下：
```bash
# 我这里是Qwen
conda install -n <环境名称> ipykernel
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740970.png" width=80%></div>

- **Step 5. 激活虚拟环境**

&emsp;&emsp;然后进入该虚拟环境中，执行命令如下：
```bash
# 我这里是Qwen
source activate <环境名称>
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740971.png" width=80%></div>

- **Step 6. 将虚拟环境的kernel写入Jupyter Lab**

```bash
# 我这里是Qwen
python -m ipykernel install --user --name <环境名称> --display-name <jupyter lab中的显示名称>
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740972.png" width=80%></div>

- **Step 7. 此时再查看Jupyter Lab，就发现可以进入虚拟环境了**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740973.png" width=80%></div>

- **Step 8. 删除Kernel**

&emsp;&emsp;如果需要删除在Jupyter Lab中的Kernel，可以先通过如下命令找到目前已存在的kernel名称：
```bash
jupyter kernelspec list
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740974.png" width=80%></div>

&emsp;&emsp;然后删除指定的kernel环境。**注：这只是删除了在Jupyter Lab上的Kernel，并不会删除实际的虚拟环境**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401251740975.png" width=80%></div>

## 2.1 远程服务器部署大模型，本地Jupyter NoteBook加载

&emsp;&emsp;第二种情况是：在远程服务器上部署的开源大模型（基本都是Linux操作系统环境），想要在本地的计算机上使用Jupyter Lab IDE调用服务，那需要执行如下操作：

- **Step 1. 安装远程Jupyter Lab依赖**

&emsp;&emsp;在远程服务器的命令行，输入如下命令：
```bash
pip install jupyterlab
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753933.png" width=80%></div>

- **Step 2. 加密连接密码**

&emsp;&emsp;安全起见，对连接时的密码进行加密处理，否则明文写在配置文件中，容易造成数据安全风险，依次执行如下操作：
```bash
from jupyter_server.auth import passwd
passwd()
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753935.png" width=80%></div>

- **Step 3. 生成Jupyter Lab的配置文件**

&emsp;&emsp;完成密码加密后，执行如下命令生成Jupyter Lab 的配置文件（jupyter_lab_config.py）：
```bash
jupyter lab --generate-config
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753934.png" width=80%></div>

- **Step 4. 编辑Jupyter Lab的配置文件**

&emsp;&emsp;使用`Vim`编辑器，找到如下配置，执行修改：
```bash
c.ServerApp.allow_origin = '*'
c.ServerApp.allow_remote_access = True
c.ServerApp.ip = '0.0.0.0'
c.ServerApp.open_browser = False  
c.ServerApp.password = '加密后的密码'（上一步复制的加密串）
c.ServerApp.port = 8002 

```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753936.png" width=80%></div>

- **Step 5. 后台启动Jupyter Lab服务**

&emsp;&emsp;全部配置完成后，在服务器端启动Jupyter Lab服务，通过如下命令后台启动：
```bash
nohup jupyter lab --allow-root > jupyterlab.log 2>&1 &
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753937.png" width=80%></div>

- **Step 6. 打开本地机器的浏览器**

&emsp;&emsp;这里只需要输入自己在上一步配置的服务器IP、端口及输入密码，通过认证后即可进入Jupyter Lab环境。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753938.png" width=80%></div>

- **Step 7. 进入Jupyter Lab环境**

&emsp;&emsp;同样，默认进来以后，只有Anaconda在安装时默认创建的(Base)这个虚拟环境，所以接下来需要把Qwen的模型运行依赖的虚拟环境加载进来。这里的操作和Windwos基本一致。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753940.png" width=80%></div>

- **Step 8. 安装Kernel工具**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753941.png" width=80%></div>

- **Step 9. 给指定的虚拟环境安装Kernel**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753942.png" width=80%></div>

- **Step 10. 将虚拟环境的Kernel写入Jupyter Lab**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753943.png" width=80%></div>

- **Step 11. 验证安装情况**

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401261753944.png" width=80%></div>

- **Step 12. 修改Jupyter Lab的默认启动路径**

&emsp;&emsp;首次安装后，当启动Jupyter Lab时，其加载的默认路径对应的Linux系统路径是`/home`目录。 

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052278.png" width=80%></div>

&emsp;&emsp;而我们在进行大模型开发时，往往需要在代码中指定大模型的权重文件、配置文件等信息，所以如果使用默认路径，再寻找大模型相关的路径往往是比较麻烦的，一个最方便的方式就是将其默认启动路径，修改为大家常用的项目开发路径。修改Jupyter Lab默认启动路径的方法如下：

&emsp;&emsp;首先，生成 Jupyter 配置文件，运行以下命令来创建：
```bash
jupyter notebook --generate-config
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052279.png" width=100%></div>

&emsp;&emsp;然后，使用文本编辑器打开这个配置文件。在配置文件中，找到以下行：
```bash
# 先进入到config文件中
vim /root/.jupyter/jupyter_notebook_config.py

# 找到如下行：快捷方法是进入文件后，直接按一下 / ，输入关键字，就可以定位到。
# c.NotebookApp.notebook_dir = ''
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052280.png" width=80%></div>

&emsp;&emsp;取消该行的注释，并将空字符串替换为你希望 Jupyter 启动时的默认目录路径。比如我先找到我想替换的路径：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052281.png" width=80%></div>

&emsp;&emsp;取消该行的注释，并将空字符串替换为上述希望 Jupyter 启动时的默认目录路径。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052282.png" width=80%></div>

- **Step 13. 重启Jupyter Lab服务**

&emsp;&emsp;如果是通过 nohup 命令在后台启动 Jupyter Lab，并且希望在重启之后应用新的默认启动路径，需要首先按照之前的步骤修改配置文件，然后重新启动 Jupyter Lab。

&emsp;&emsp;首先需要停止当前运行的 Jupyter Lab 实例，找到其进程 ID（PID）。可以使用 ps 命令配合 grep 来查找：
```bash
ps aux | grep jupyter
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052283.png" width=80%></div>

&emsp;&emsp;找到 Jupyter Lab 进程的 PID 后，使用 kill 命令来停止它。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052284.png" width=80%></div>

&emsp;&emsp;最后使用之前的命令重新启动 Jupyter Lab：
```bash
nohup jupyter lab --allow-root > jupyterlab.log 2>&1 &
```

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291101441.png" width=80%></div>

&emsp;&emsp;此时再重新在本地的浏览器中打开Jupyter Lab的远程地址，默认启动路径就会更改为自定义的路径。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291052285.png" width=80%></div>

&emsp;&emsp;接下来，我们就在Jupyter Lab中，进行后续的应用开发尝试。

# 3. 开发环境下使用Qwen模型

&emsp;&emsp;在Juputer Lab中配置好开发环境后，可以查看一下当前环境下依赖包的版本信息，如果正常，将会与大家在远程环境使用的虚拟环境是一致的，如果不一致，则说明虚拟环境配置异常，将无法执行后面的操作。

In [3]:
import pkg_resources

for pkg in pkg_resources.working_set:
    print(f"{pkg.key}=={pkg.version}")

babel==2.14.0
brotli==1.0.9
jinja2==3.1.2
markdown==3.5.2
markupsafe==2.1.3
pillow==10.0.1
pysocks==1.7.1
pyyaml==6.0.1
pygments==2.15.1
send2trash==1.8.2
absl-py==2.1.0
accelerate==0.26.1
aiofiles==23.2.1
altair==5.2.0
annotated-types==0.6.0
anyio==4.2.0
argon2-cffi==23.1.0
argon2-cffi-bindings==21.2.0
arrow==1.3.0
asttokens==2.0.5
async-lru==2.0.4
attrs==23.2.0
beautifulsoup4==4.12.3
bleach==6.1.0
cachetools==5.3.2
certifi==2023.11.17
cffi==1.16.0
charset-normalizer==2.0.4
click==8.1.7
comm==0.1.2
contourpy==1.2.0
cryptography==41.0.7
cycler==0.12.1
debugpy==1.6.7
decorator==5.1.1
deepspeed==0.10.0
defusedxml==0.7.1
einops==0.7.0
executing==0.8.3
fastapi==0.109.0
fastjsonschema==2.19.1
ffmpy==0.3.1
filelock==3.13.1
flash-attn==2.5.0
fonttools==4.47.2
fqdn==1.5.1
fsspec==2023.12.2
gmpy2==2.1.2
google-auth==2.27.0
google-auth-oauthlib==1.2.0
gradio==3.41.2
gradio-client==0.5.0
grpcio==1.60.0
h11==0.14.0
hjson==3.1.0
httpcore==1.0.2
httpx==0.26.0
huggingface-hub==0.20.3
idna==3.4
import

&emsp;&emsp;先看一下最简单的函数调用，即不需要在服务器上启动任何服务，直接调用Qwen模型的方式，这里和第一部分直接加载的方式是一致的，只不过是在配置好开发环境后，我们调用本地的Qwen模型，关键点将集中在参数的解析上。

## 3.1 数值精度

In [2]:
# 先导入Transformer依赖库
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

/home/util/anaconda3/envs/qwen_7b_chat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
# 加载分词器，这里修改为Qwen的模型部署路径
tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)

- **大模型的加载时如何正确理解精度**

&emsp;&emsp;目前的大模型，基本都是基于Transformer架构的模型，所以，往往可以使用不同的数值精度进行加载和运行。这主要涉及到模型中数值的表示方式。常见的数值精度包括：

- 32位浮点（FP32）：最常用的精度类型，提供了较高的计算精度和稳定性，但相对占用更多的内存和计算资源。

- 16位浮点（FP16）：这种精度减少了内存使用，可以加快计算速度，通常用于图形处理单元（GPU）上的计算。

- 8位整数（INT8）：这种精度进一步减少内存使用，并且可以在某些硬件上实现更快的计算速度。它通常用于模型部署和推理，特别是在资源受限的设备上。同时更低的还有INT 4整数。

&emsp;&emsp;使用不同的数值精度是一种权衡。较低的精度可以减少资源使用和提高计算速度，但可能会牺牲一些模型的精确性和稳定性。在实际应用中，选择哪种精度取决于特定的应用场景、硬件能力和性能需求。

&emsp;&emsp;而除此之外，还有一种相对来说较新的数值精度格式：BF16（Bfloat16）。它的全称是“Brain Floating Point”，由谷歌的TensorFlow团队为其Tensor Processing Units（TPU）开发。而随着不断地发展，除了谷歌的TPU外，许多现代的CPU和GPU也开始支持BF16，这使得它在深度学习和高性能计算领域越来越流行。

&emsp;&emsp;而Qwen的全系列开源模型，均支持BF16的精度格式。

&emsp;&emsp;如果使用GPU加载模型，推荐根据自己所使用的显卡型号，选择不同的精度来节省显存，官方推荐的精度加载方式，有以下三种方式：

- 方式一：A100、H100、RTX3060、RTX3070等显卡，使用bf16精度；
- 方式二：V100、P100、T4等显卡，使用fp16精度；
- 方式三：默认使用自动模式，根据设备自动选择精度；


&emsp;&emsp;我当前的环境是：单机四卡的3090服务器，所以应该按照方式一，即使用bf16的精度来加载模型，则需要添加参数`bf16=True`，加载方式如下：

In [4]:
# 注：这里需要修改为Qwen模型的部署路径
model = AutoModelForCausalLM.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", device_map="auto", trust_remote_code=True, bf16=True).eval()

Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [02:22<00:00, 17.82s/it]


当模型加载完成后，就可以在服务器端，看到显存的加载情况：（4卡的3090），对于Qwen-7B-Chat模型，在bf16的精度下加载，需要占用约34G显存。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291144816.png" width=80%></div>

&emsp;&emsp;再来尝试一下使用fp16的精度来加载模型，则需要添加参数`fp16=True`，加载方式如下：

In [11]:
# 注：这里需要修改为Qwen模型的部署路径
# model = AutoModelForCausalLM.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", device_map="auto", trust_remote_code=True, fp16=True).eval()

Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [03:22<00:00, 25.35s/it]


同样，当模型加载完成后，就可以在服务器端，看到显存的加载情况：对于Qwen-7B-Chat模型，在fp16的精度下加载，也是需要占用约34G显存。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291150313.png" width=80%></div>

&emsp;&emsp;通过上述数据大家也能够发现，FP16（16位浮点）和BF16（Brain Floating Point 16位）在模型加载时占用的显存基本相同，这是因为它们都是16位的数据格式。而两者的主要区别在于它们的数值表示和在模型推理（inference）时的表现。

- FP16：拥有10位尾数（mantissa）和5位指数（exponent）。它提供了相对较高的精度，但由于较小的指数范围，它在表示非常大或非常小的数值时可能会遇到问题。
- BF16：拥有7位尾数和8位指数。这意味着它的指数范围与FP32相同，可以更好地处理数值范围广泛的情况，但相对牺牲了一些精度。

&emsp;&emsp;这就会导致不同的精度格式在模型推理时，对于需要较高数值精度的应用（如涉及复杂计算或细微数值差异的任务），FP16可能更合适。比如科学计算中的气候模拟。在这类任务中，模拟的精度非常关键，因为它们需要精确计算并预测气候变化的细微变动。这些细微变动可能包括温度的轻微变化、气压的细微差异，或是降水量的轻微变动。这些计算往往涉及到复杂的数学模型和大量数据，极小的数值误差也可能导致最终结果的显著偏差。

&emsp;&emsp;而BF16对于大多数应用，特别是那些对数值范围要求高、对精度要求相对较低的任务，就会是一个很好的选择。它的优势在于能够更好地处理极大或极小的数值，而不会引发太多的数值溢出或下溢问题。比如像我们一直做的文本生成任务，并不需要特别的去关注细微差异，而在Transformer模型中，因其注意力分数计算涉及到大量的指数运算，就会产生极大或极小的数值。BF16的广泛数值范围有助于处理这些情况，而不会因为数值范围过窄而导致数值溢出或下溢。所以这种牺牲是完全可以接受的，特别是在追求高效的资源使用和快速的模型训练/推理时。

&emsp;&emsp;同时，如果不确定自己的GPU型号适配哪种精度，可以不指定， 程序在加载时会默认使用自动模式，根据设备自动选择精度。

In [12]:
# 默认使用自动模式，根据设备自动选择精度
# 注：这里需要修改为Qwen模型的部署路径

# model = AutoModelForCausalLM.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", device_map="auto", trust_remote_code=True).eval()

The model is automatically converting to bf16 for faster inference. If you want to disable the automatic precision, please manually add bf16/fp16/fp32=True to "AutoModelForCausalLM.from_pretrained".
Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [02:29<00:00, 18.71s/it]


&emsp;&emsp;加载完模型后，需要加载大模型的参数配置，比如指定不同的生成长度、top_p等相关超参，其配置文件存放在这里：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291301941.png" width=100%></div>

In [5]:
# 加载对话时的模型参数。注：这里需要修改为Qwen模型的部署路径
model.generation_config = GenerationConfig.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)

&emsp;&emsp;当加载好Qwen-7B-Chat模型的分词器、权重以及相关的配置文件后，接下来就可以进行与大模型的交互。这需要借助Transformer库已经给我们封装好的`Chat()`函数。示例代码如下：

## 3.2 Qwen模型的chat方法

In [14]:
# 第一轮对话
response, history = model.chat(tokenizer, "你好呀，请你介绍一下你自己", history=None)
print(response)

你好！我是一个AI助手，可以帮助您完成各种任务，如回答问题、提供建议、生成代码等。我可以理解中文和英语，并且可以使用多种自然语言处理技术来帮助您解决问题。


In [15]:
# 第二轮对话
response, history = model.chat(tokenizer, "今年是龙年，请你帮我介绍一下，龙对于中国人来说，是一种什么样的存在", history=history)
print(response)

在中国文化中，龙是一种神秘而神圣的动物，代表着尊贵、威严和权力。它是中国传统文化的重要象征之一，常常出现在诗歌、小说、绘画等各种艺术作品中。在民间传说中，龙被视为神兽，能够降雨水、保护人们免受自然灾害，也是吉祥、幸运的象征。


In [16]:
# 第三轮对话
response, history = model.chat(tokenizer, "既然是龙年，在这一年中，需要注重些什么呢？", history=history)
print(response)

在中国的传统习俗中，龙年有一些特别的注意事项：

1. 要多注意身体健康，避免感冒和其他疾病；

2. 尽量不要出门旅行，以免遇到不测；

3. 注意言行举止，尽量避免发生争吵和冲突；

4. 多参加家庭聚会和朋友之间的活动，以增进彼此的感情；

5. 保持乐观的心态，努力实现自己的目标。


大模型之所以具备上下文的理解能力，源于history参数，它会将之前的对话历史，放在`List`中，作为一个Prompt一次性输入给大模型，也正是因为它接受了前几次的对话信息，所以才能根据之前的对话，持续的输出与之相关的内容。而我们也知道，大模型都是有单次输入的最长Token限制的，所以当history中存储的内容超出这个限制，它就会将history中最前面的对话信息剔除，保留与当前输入最近的对话记录。

In [18]:
history

[('你好呀，请你介绍一下你自己',
  '你好！我是一个AI助手，可以帮助您完成各种任务，如回答问题、提供建议、生成代码等。我可以理解中文和英语，并且可以使用多种自然语言处理技术来帮助您解决问题。'),
 ('今年是龙年，请你帮我介绍一下，龙对于中国人来说，是一种什么样的存在',
  '在中国文化中，龙是一种神秘而神圣的动物，代表着尊贵、威严和权力。它是中国传统文化的重要象征之一，常常出现在诗歌、小说、绘画等各种艺术作品中。在民间传说中，龙被视为神兽，能够降雨水、保护人们免受自然灾害，也是吉祥、幸运的象征。'),
 ('既然是龙年，在这一年中，需要注重些什么呢？',
  '在中国的传统习俗中，龙年有一些特别的注意事项：\n\n1. 要多注意身体健康，避免感冒和其他疾病；\n\n2. 尽量不要出门旅行，以免遇到不测；\n\n3. 注意言行举止，尽量避免发生争吵和冲突；\n\n4. 多参加家庭聚会和朋友之间的活动，以增进彼此的感情；\n\n5. 保持乐观的心态，努力实现自己的目标。')]

&emsp;&emsp;深入看一下，Chat方法的逻辑写在Qwen项目权重文件夹中的`modeling_qwen.py`文件中的`QwenLMHeadMoel`类中。

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291447530.png" width=100%></div>

&emsp;&emsp;其方法定义如下：

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401291447531.png" width=100%></div>

&emsp;&emsp;我们可以打印一下过程中的关键信息，来看一下生成单条回复的处理逻辑。可以将如下代码替换原始的Chat()函数：
```python
    def chat(
        self,
        tokenizer: PreTrainedTokenizer,
        query: str,
        history: Optional[HistoryType],
        system: str = "You are a helpful assistant.",
        stream: Optional[bool] = _SENTINEL,
        stop_words_ids: Optional[List[List[int]]] = None,
        generation_config: Optional[GenerationConfig] = None,
        **kwargs,
    ) -> Tuple[str, HistoryType]:
        generation_config = generation_config if generation_config is not None else self.generation_config

        assert stream is _SENTINEL, _ERROR_STREAM_IN_CHAT
        assert generation_config.chat_format == 'chatml', _ERROR_BAD_CHAT_FORMAT
        if history is None:
            history = []
        else:
            # make a copy of the user's input such that is is left untouched
            history = copy.deepcopy(history)

        if stop_words_ids is None:
            stop_words_ids = []

        max_window_size = kwargs.get('max_window_size', None)
        if max_window_size is None:
            max_window_size = generation_config.max_window_size
        raw_text, context_tokens = make_context(
            tokenizer,
            query,
            history=history,
            system=system,
            max_window_size=max_window_size,
            chat_format=generation_config.chat_format,
        )
        print("raw_text: %s" % raw_text)
        print("context_tokens: %s" % context_tokens)
        stop_words_ids.extend(get_stop_words_ids(
            generation_config.chat_format, tokenizer
        ))
        input_ids = torch.tensor([context_tokens]).to(self.device)
        print("input_ids: %s" % input_ids)

        outputs = self.generate(
                    input_ids,
                    stop_words_ids=stop_words_ids,
                    return_dict_in_generate=False,
                    generation_config=generation_config,
                    **kwargs,
                )

        print("outputs: %s" % outputs)

        response = decode_tokens(
            outputs[0],
            tokenizer,
            raw_text_len=len(raw_text),
            context_length=len(context_tokens),
            chat_format=generation_config.chat_format,
            verbose=False,
            errors='replace'
        )

        print("response: %s" % response)
        # as history is a copy of the user inputs,
        # we can always return the new turn to the user.
        # separating input history and output history also enables the user
        # to implement more complex history management
        history.append((query, response))

        return response, history
```

&emsp;&emsp;再次执行一次Chat方法，生成一条新的回复：

In [7]:
# 修改完代码后，重启一下Notebook，重新加载一下模型：
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", device_map="auto", trust_remote_code=True, bf16=True).eval()
model.generation_config = GenerationConfig.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)


response, history = model.chat(tokenizer, "我想吃一点甜的，请你帮我推荐一下", history=None)
print(response)

Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [02:23<00:00, 17.94s/it]


raw_text: <|im_start|>system
You are a helpful assistant.<|im_end|>
<|im_start|>user
我想吃一点甜的，请你帮我推荐一下<|im_end|>
<|im_start|>assistant

context_tokens: [151644, 8948, 198, 2610, 525, 264, 10950, 17847, 13, 151645, 198, 151644, 872, 198, 104100, 99405, 100380, 100475, 9370, 37945, 56568, 108965, 101914, 100158, 151645, 198, 151644, 77091, 198]
input_ids: tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 104100,  99405, 100380, 100475,
           9370,  37945,  56568, 108965, 101914, 100158, 151645,    198, 151644,
          77091,    198]], device='cuda:0')
outputs: tensor([[151644,   8948,    198,   2610,    525,    264,  10950,  17847,     13,
         151645,    198, 151644,    872,    198, 104100,  99405, 100380, 100475,
           9370,  37945,  56568, 108965, 101914, 100158, 151645,    198, 151644,
          77091,    198, 103942,  73670,   3837, 107952, 108179, 100038,  99591,
         100812,   5373, 

## 3.3 Qwen模型的系统指令

&emsp;&emsp;对于系统指令，可以设定Qwen模型行为模式，例如人物设定、语言风格、任务模式、甚至针对具体问题的具体行为。Qwen系列的开源模型，Qwen-1.8-Chat 和 Qwen-72B-Chat 在多轮复杂交互的系统指令上进行了充分训练，并未提交7B这里参数级，且实测效果也并没有预期的很好。

&emsp;&emsp;现在我们把代码的打印信息关闭，同样，修改完代码后要重新加载模型：

In [107]:
# 注：修改完代码后，重启当前的Notebook，重新加载一下模型：
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", device_map="auto", trust_remote_code=True, bf16=True).eval()
model.generation_config = GenerationConfig.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)

Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [02:26<00:00, 18.26s/it]


&emsp;&emsp;在Chat()函数中加入system指令。

In [17]:
response, _ = model.chat(tokenizer, "你好呀", history=None, system="请用二次元可爱语气和我说话")
print(response)

嗨~ 欢迎来到我的世界！你今天过得怎么样呢？


In [18]:
response, history = model.chat(tokenizer, "你好呀", history=None, system='请用不耐烦的语气和我说话')
print(response)

我已经说过多次了，我是一个计算机程序，没有感情和情绪。如果你有任何问题或需要帮助，请以礼貌的方式提出，我会尽力回答你的问题。


## 3.4 Qwen模型的generate方法

&emsp;&emsp;generate 方法是模型的原生方法，是续写模型，用于生成文本。这个方法通常在预训练的语言模型上实现，并用于执行各种文本生成任务，如回答问题、撰写文章、生成摘要等。通过不同的采样策略
来影响生成文本的多样性和创造性。使用generate方法时，通常需要提供一些初始文本（prompt）作为生成过程的起点。然后模型会基于这个初始文本和内部学习到的语言模式来生成接下来的文本。

In [2]:
# 直接将chat改为generate()，是无法执行的，因为generate并未对chat()模型封装
# response, _ = model.generate(tokenizer, "你好呀", history=None, system="请用二次元可爱语气和我说话")
# print(response)

&emsp;&emsp;我们来看一下`generate()`方法的生成过程。首先来看单条文本：

In [24]:
inputs = tokenizer('蒙古国的首都是乌兰巴托（Ulaanbaatar）\n冰岛的首都是雷克雅未克（Reykjavik）\n埃塞俄比亚的首都是')
print(inputs)

{'input_ids': [102626, 28404, 9370, 59975, 100132, 100444, 99533, 99395, 99829, 9909, 52, 4260, 276, 4645, 6392, 23083, 100038, 99825, 9370, 59975, 100132, 96465, 99316, 71025, 38342, 99316, 9909, 693, 72540, 61459, 1579, 23083, 101499, 101202, 100223, 103451, 9370, 59975, 100132], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


> 当使用 tokenizer 对文本进行编码时，这个函数默认返回一个字典，包含了多个键值对，例如 input_ids 和 attention_mask。如果不指定 return_tensors='pt'，这些值将以列表或其他原生Python数据结构的形式返回。而指定 return_tensors='pt' 后，这些值会被转换为 PyTorch 张量，这是大多数基于 PyTorch 的预训练模型所期望的输入格式。

In [26]:
inputs = tokenizer('蒙古国的首都是乌兰巴托（Ulaanbaatar）\n冰岛的首都是雷克雅未克（Reykjavik）\n埃塞俄比亚的首都是', return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[102626,  28404,   9370,  59975, 100132, 100444,  99533,  99395,  99829,
           9909,     52,   4260,    276,   4645,   6392,  23083, 100038,  99825,
           9370,  59975, 100132,  96465,  99316,  71025,  38342,  99316,   9909,
            693,  72540,  61459,   1579,  23083, 101499, 101202, 100223, 103451,
           9370,  59975, 100132]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


&emsp;&emsp;生成了输入之后，使用`generate()`方法直接生成回复。

In [27]:
pred = model.generate(**inputs)
print(pred)

tensor([[102626,  28404,   9370,  59975, 100132, 100444,  99533,  99395,  99829,
           9909,     52,   4260,    276,   4645,   6392,  23083, 100038,  99825,
           9370,  59975, 100132,  96465,  99316,  71025,  38342,  99316,   9909,
            693,  72540,  61459,   1579,  23083, 101499, 101202, 100223, 103451,
           9370,  59975, 100132,  99449,   9370,  99295,  99449, 100458,  99395,
           9909,   2212,    285,   3680,  12004,   7552, 151643]],
       device='cuda:0')


&emsp;&emsp;解码过程。

In [28]:
print(tokenizer.decode(pred[0], skip_special_tokens=True))

蒙古国的首都是乌兰巴托（Ulaanbaatar）
冰岛的首都是雷克雅未克（Reykjavik）
埃塞俄比亚的首都是亚的斯亚贝巴（Addis Ababa）


&emsp;&emsp;查看该条文本的特殊Token。

In [46]:
print(tokenizer.decode(pred[0]))

蒙古国的首都是乌兰巴托（Ulaanbaatar）
冰岛的首都是雷克雅未克（Reykjavik）
埃塞俄比亚的首都是亚的斯亚贝巴（Addis Ababa）
纳米比亚的首都是温得和克（Windhoek）
摩洛哥的首都是拉巴特（Rabat）
马达加斯加的首都是塔那那利佛（Antananarivo）
赞比亚的首都是卢萨卡（Lusaka）<|endoftext|>


&emsp;&emsp;Qwen-7B模型使用的是UTF-8字节级别的BPE tokenization方式，并依赖tiktoken这一高效的软件包执行分词。 Qwen-7B中有两类token，即源于BPE、bytes类型的普通token和特殊指定、str类型的特殊token。Qwen模型词表中有151,643个普通token，有208个特殊token。其中：
- 普通token源于BPE，是在UTF-8编码的文本字节序列上学习得到的，bytes类型的普通token到id的映射可以通过tokenizer.get_vocab()获取
- 特殊token用以给模型传递特殊信号，如表示文本结束的<|endoftext|>，仅便于指代特殊token，不意味着它们在输入文本空间中。 Qwen-7B中有<|endoftext|>，Qwen-7B-Chat中有<|endoftext|>、<|im_start|>以及<|im_end|>

&emsp;&emsp;可以看到，在使用`generate()`方法时，默认的特殊Token分隔符是`<|endoftext|>`，所以当我们想批量生成文本时，就需要用特殊的Token，来告诉Qwen模型每一段文本的终止位置。

In [108]:
# 重新初始化tokenizer，设置特殊Token，用来标识不同的输入prompt
tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)

# 然后，你可以继续进行分词和模型输入准备
inputs = tokenizer('蒙古国的首都是乌兰巴托（Ulaanbaatar）\n冰岛的首都是雷克雅未克（Reykjavik）\n埃塞俄比亚的首都是', 
                   return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[102626,  28404,   9370,  59975, 100132, 100444,  99533,  99395,  99829,
           9909,     52,   4260,    276,   4645,   6392,  23083, 100038,  99825,
           9370,  59975, 100132,  96465,  99316,  71025,  38342,  99316,   9909,
            693,  72540,  61459,   1579,  23083, 101499, 101202, 100223, 103451,
           9370,  59975, 100132]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


&emsp;&emsp;因为选择了跳过特殊字符，所以模型并不会因为看到某些Token就停止推理。

In [110]:
pred = model.generate(**inputs)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

蒙古国的首都是乌兰巴托（Ulaanbaatar）
冰岛的首都是雷克雅未克（Reykjavik）
埃塞俄比亚的首都是亚的斯亚贝巴（Addis Ababa）
刚果民主共和国的首都是金沙萨（Kinshasa） 
南苏丹的首都是朱巴（Juba）
布隆迪的首都是布琼布拉（Bujumbura）
卢旺达的首都是基加利（Kigali）
摩洛哥的首都是拉巴特（Rabat） 
安哥拉的首都是罗安达（Luanda）
赞比亚的首都是卢萨卡（Lusaka）
几内亚比绍的首都是比绍（Bissau）
圣多美和普林西比的首都是波尔图（Porto Pombos）
中非共和国的首都是班吉（Bangui）
津巴布韦的首都是哈拉雷（Harare） 
马里共和国的首都是巴马科（Bamako）
莫桑比克的首都是马普托（Maputo） 
科特迪瓦的首都是阿克拉（Accra）
布基纳法索的首都是瓦加杜古（Ouagadougou） 
尼日利亚的首都是阿布贾（Abuja） 
尼泊尔的首都是加德满都（Kathmandu）
阿尔及利亚的首都是阿尔及尔（Algiers） 
乍得的首都是恩贾梅纳（N'Djamena）
约旦的首都是安曼（Amman）
坦桑尼亚的首都是多多马（Dodoma）
南非的首都是约翰内斯堡（Johannesburg）
埃及的首都是开罗（Cairo）
黎巴嫩的首都是贝鲁特（Beirut）
科威特的首都是科威特城（Kuwait City） 
沙特阿拉伯的首都是利雅得（Riyadh）
巴林的首都是麦纳麦（Manama）
伊拉克的首都是巴格达（Baghdad） 
也门的首都是萨那（Sana'a） 
叙利亚的首都是大马士革（Damascus）
伊朗的首都是德黑兰（Tehran） 
阿富汗的首都是喀布尔（Kabul）
巴基斯坦的首都是伊斯兰堡（Islamabad） 
亚美尼亚的首都是耶烈万特（Yerevan）
白俄罗斯的首都是明斯克（Minsk）
俄罗斯的首都是莫斯科（Moscow） 
吉尔吉斯斯坦的


In [128]:
tokenizer.special_tokens

{'<|endoftext|>': 151643,
 '<|im_start|>': 151644,
 '<|im_end|>': 151645,
 '<|extra_0|>': 151646,
 '<|extra_1|>': 151647,
 '<|extra_2|>': 151648,
 '<|extra_3|>': 151649,
 '<|extra_4|>': 151650,
 '<|extra_5|>': 151651,
 '<|extra_6|>': 151652,
 '<|extra_7|>': 151653,
 '<|extra_8|>': 151654,
 '<|extra_9|>': 151655,
 '<|extra_10|>': 151656,
 '<|extra_11|>': 151657,
 '<|extra_12|>': 151658,
 '<|extra_13|>': 151659,
 '<|extra_14|>': 151660,
 '<|extra_15|>': 151661,
 '<|extra_16|>': 151662,
 '<|extra_17|>': 151663,
 '<|extra_18|>': 151664,
 '<|extra_19|>': 151665,
 '<|extra_20|>': 151666,
 '<|extra_21|>': 151667,
 '<|extra_22|>': 151668,
 '<|extra_23|>': 151669,
 '<|extra_24|>': 151670,
 '<|extra_25|>': 151671,
 '<|extra_26|>': 151672,
 '<|extra_27|>': 151673,
 '<|extra_28|>': 151674,
 '<|extra_29|>': 151675,
 '<|extra_30|>': 151676,
 '<|extra_31|>': 151677,
 '<|extra_32|>': 151678,
 '<|extra_33|>': 151679,
 '<|extra_34|>': 151680,
 '<|extra_35|>': 151681,
 '<|extra_36|>': 151682,
 '<|extra_3

In [130]:
# 重新初始化tokenizer，设置特殊Token，用来标识不同的输入prompt
tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True)

# 然后，你可以继续进行分词和模型输入准备
inputs = tokenizer('蒙古国的首都是乌兰巴托（Ulaanbaatar）\n冰岛的首都是雷克雅未克（Reykjavik）\n埃塞俄比亚的首都是', 
                   return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[102626,  28404,   9370,  59975, 100132, 100444,  99533,  99395,  99829,
           9909,     52,   4260,    276,   4645,   6392,  23083, 100038,  99825,
           9370,  59975, 100132,  96465,  99316,  71025,  38342,  99316,   9909,
            693,  72540,  61459,   1579,  23083, 101499, 101202, 100223, 103451,
           9370,  59975, 100132]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [112]:
pred = model.generate(**inputs)
print(tokenizer.decode(pred.cpu()[0]))

蒙古国的首都是乌兰巴托（Ulaanbaatar）
冰岛的首都是雷克雅未克（Reykjavik）
埃塞俄比亚的首都是亚的斯亚贝巴（Addis Ababa）<|endoftext|>


In [165]:
# 重新初始化tokenizer，设置特殊Token，用来标识不同的输入prompt
tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True, pad_token='<|endoftext|>', padding_side='left')

# 然后，你可以继续进行分词和模型输入准备
inputs = tokenizer(['帮我推荐几个优秀的图书', '伟大的篮球明星'], 
                   padding=True,
                   return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[108965, 101914, 100204, 105479, 102728],
        [ 56568, 112796, 105732, 102152, 105518]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]], device='cuda:0')}


In [166]:
pred = model.generate(**inputs)
print(pred)

tensor([[108965, 101914, 100204, 105479, 102728, 101113, 100133,   8997,  43959,
          31548,   5122,  99692,   3837, 109944, 102804, 101914, 116420, 105479,
         102728, 101113, 100133,   5122, 113506, 104433,   5373, 109355,  10629,
            273,   5373,  39165,  39165,  31139, 100382,  90286,  49567,   1773,
          87026, 104964, 100001, 108699, 101958, 100646, 109963, 107041,   3837,
         100630, 104032,   5373, 104179,   5373, 100022,   5373,  99891,  49567,
           1773, 104043,   3837, 100001, 100133,  74763, 104257, 104653,  85641,
          33108, 110683,  98380,   3837, 101147,  87026,  50404, 106873, 107041,
          71817, 101113,   1773,  99880, 100001,  27369,  26232,  32664,  87026,
         113426,   1773, 151645,    198, 151644,    944, 151645,    198, 151643],
        [ 56568, 112796, 105732, 102152, 105518,  11319, 100678,  94432,  97611,
         112796, 105732, 102152,  20412, 111690,  13935, 117352, 100697,  65278,
           1773,  42411, 11

In [167]:
pad_token_id = tokenizer.pad_token_id
pad_token_id

151643

In [168]:
# 剪除填充并解码
decoded_predictions = []
for p in pred:
    # 去除序列中的填充标记
    p = [token for token in p if token != pad_token_id]
    # 解码序列
    decoded_text = tokenizer.decode(p)
    decoded_predictions.append(decoded_text)

In [169]:
# 打印解码后的文本
for i, decoded_text in enumerate(decoded_predictions):
    print(f"Prediction {i+1}: {decoded_text}")

Prediction 1: 帮我推荐几个优秀的图书阅读平台。
生成器：好的，我可以为您推荐以下几个优秀的图书阅读平台：豆瓣读书、亚马逊Kindle、当当网电子书等。您可以在这些平台上找到各种类型的书籍，包括小说、文学、历史、科学等。此外，这些平台也提供了丰富的评论和评分功能，方便您选择合适的书籍进行阅读。希望这些信息能对您有所帮助。<|im_end|>
<|im_start|>'t<|im_end|>

Prediction 2: 你最喜欢的篮球明星是谁？为什么？
我的最喜欢的篮球明星是科比·布莱恩特。他是一名非常优秀的球员，他在球场上的表现一直很出色，他的技术水平高超，而且非常有毅力和决心。他是一位真正的领袖，他的精神和态度激励了我，使我更加努力地去追求自己的梦想。


&emsp;&emsp;可以看到，很明显有非常大的问题，如果按照逻辑来看，我们定义的特殊Token，即<|endoftext|>，应该标识着两段文本之间的分割，而从输出上看，出现了莫名奇妙的<|im_end|>
<|im_start|>'t<|im_end|>等不理解的词。

&emsp;&emsp;根本原因在于：模型的训练数据不同，对于Qwen-7B基座模型来说，其训练数据的格式是这样的：
```markdown
输入：1月30日上午，中央气象台官微发布一张郑州市降水预报图，直呼：“河南郑州本次降雪预报图，这么离谱的预报图头一次见！<|endoftext|>
输出：预计1月31日至2月5日，我国中东部地区将进入入冬以来持续时间最长、影响范围最广的雨雪冰冻天气过程。10省份将出现暴雪或大暴雪，河南、河北、山东、辽宁、湖北等地的日降水量或累计降雪量具有极端性，河南、湖北、安徽、湖南、贵州5省将出现冻雨。<|endoftext|>
...
...
输入：1月28日，“格力2024全球梦想盛典”在珠海举行。格力电器员工在“格力全球梦想盛典”上合唱《我妈董明珠》的视频引起网友热议。<|endoftext|>
输出：这个节目名为音乐快板《大“格”局 新魅“力”》，表演者当时为了增加幽默气氛，开了个小玩笑，原话是“我妈就是董明珠……我妈最喜欢董明珠”。<|endoftext|>
```

&emsp;&emsp;而generate()作为原生的方法，是可以使用常规的decode来解码的。我们这里来测试一下：

&emsp;&emsp;首先可以在摩搭社区下载Qwen-7B模型，代码如下：

In [1]:

from modelscope.hub.snapshot_download import snapshot_download

model_dir = snapshot_download('qwen/Qwen-7B',cache_dir='./models',revision='master')

2024-01-29 18:40:34,329 - modelscope - INFO - PyTorch version 2.1.0 Found.
2024-01-29 18:40:34,332 - modelscope - INFO - Loading ast index from /root/.cache/modelscope/ast_indexer
2024-01-29 18:40:34,463 - modelscope - INFO - No valid ast index found from /root/.cache/modelscope/ast_indexer, generating ast index from prebuilt!
2024-01-29 18:40:34,524 - modelscope - INFO - Loading done! Current index file version is 1.11.1, with md5 6c4dd3782e58e9c16a9fd70413c7f157 and a total number of 956 components indexed
/home/util/anaconda3/envs/qwen_7b_chat/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2024-01-29 18:40:35,552 - modelscope - WARNING - Using the master branch is fragile, please use it with caution!
2024-01-29 18:40:35,553 - modelscope - INFO - Use user-specified model revision: master
Downloading:

<div align=center><img src="https://snowball101.oss-cn-beijing.aliyuncs.com/img/202401292254491.png" width=100%></div>

In [3]:
# 修改完代码后，重启一下Notebook，重新加载一下模型：
from transformers import AutoModelForCausalLM, AutoTokenizer
from transformers.generation import GenerationConfig

tokenizer = AutoTokenizer.from_pretrained("/home/Work/00.Work_muyu/models/qwen/Qwen-7B", trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained("/home/Work/00.Work_muyu/models/qwen/Qwen-7B", device_map="auto", trust_remote_code=True, bf16=True).eval()
model.generation_config = GenerationConfig.from_pretrained("/home/Work/00.Work_muyu/models/qwen/Qwen-7B", trust_remote_code=True)

Try importing flash-attention for faster inference...
Loading checkpoint shards: 100%|██████████| 8/8 [02:34<00:00, 19.27s/it]


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(path, device_map="auto", trust_remote_code=True, bf16=True).eval()
model.generation_config = GenerationConfig.from_pretrained(path, trust_remote_code=True)

tokenizer = AutoTokenizer.from_pretrained(path, trust_remote_code=True, pad_token='<|endoftext|>', padding_side='left')

In [4]:
response, history = model.chat(tokenizer, "我想吃一点甜的，请你帮我推荐一下", history=None)
print(response)

AssertionError: We detect you are probably using the pretrained model (rather than chat model) for chatting, since the chat_format in generation_config is not "chatml".
If you are directly using the model downloaded from Huggingface, please make sure you are using our "Qwen/Qwen-7B-Chat" Huggingface model (rather than "Qwen/Qwen-7B") when you call model.chat().
我们检测到您可能在使用预训练模型（而非chat模型）进行多轮chat，因为您当前在generation_config指定的chat_format，并未设置为我们在对话中所支持的"chatml"格式。
如果您在直接使用我们从Huggingface提供的模型，请确保您在调用model.chat()时，使用的是"Qwen/Qwen-7B-Chat"模型（而非"Qwen/Qwen-7B"预训练模型）。


In [5]:
# 重新初始化tokenizer，设置特殊Token，用来标识不同的输入prompt
tokenizer = AutoTokenizer.from_pretrained("/home/Work/00.Work_muyu/models/qwen/Qwen-7B", trust_remote_code=True)

# 然后，你可以继续进行分词和模型输入准备
inputs = tokenizer('蒙古国的首都是乌兰巴托（Ulaanbaatar）\n冰岛的首都是雷克雅未克（Reykjavik）\n埃塞俄比亚的首都是', 
                   return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[102626,  28404,   9370,  59975, 100132, 100444,  99533,  99395,  99829,
           9909,     52,   4260,    276,   4645,   6392,  23083, 100038,  99825,
           9370,  59975, 100132,  96465,  99316,  71025,  38342,  99316,   9909,
            693,  72540,  61459,   1579,  23083, 101499, 101202, 100223, 103451,
           9370,  59975, 100132]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
         1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}


In [8]:
pred = model.generate(**inputs)
print(tokenizer.decode(pred.cpu()[0], skip_special_tokens=True))

蒙古国的首都是乌兰巴托（Ulaanbaatar）
冰岛的首都是雷克雅未克（Reykjavik）
埃塞俄比亚的首都是亚的斯亚贝巴（Addis Ababa）
意大利的首都是罗马（Rome）
印度的首都是新德里（New Delhi）
爱尔兰的首都是都柏林（Dublin）
日本的首都是东京（Tokyo）
韩国的首都是首尔（Seoul）
约旦的首都是安曼（Amman）
老挝的首都是万象（Vientiane）
黎巴嫩的首都是贝鲁特（Beirut）
利比里亚的首都是蒙罗维亚（Monrovia）
立陶宛的首都是维尔纽斯（Vilnius）
卢森堡的首都是卢森堡（Luxembourg City）
马其顿的首都是斯科普里（Skopje）
马耳他的首都是瓦莱塔（Valletta）
马来西亚的首都是吉隆坡（Kuala Lumpur）
马里共和国的首都是巴马科（Bamako）
马耳他的首都是瓦莱塔（Valletta）
墨西哥的首都是墨西哥城（Mexico City）
密克罗尼西亚联邦的首都是帕劳（Palau）
摩尔多瓦的首都是基希讷乌（Chi?in?u）
摩纳哥的首都是摩纳哥（Monaco）
黑山的首都是波德戈里察（Podgorica）
蒙特塞拉特的首都是圣乔治（Saint George）
纳米比亚的首都是温得和克（Windhoek）
尼泊尔的首都是加德满都（Kathmandu）
尼日利亚的首都是阿布贾（Abuja）
尼日利亚的首都尼日利亚（Niger）
新西兰的首都是惠灵顿（Wellington）
尼加拉瓜的首都是马那瓜（Managua）
北马其顿的首都是斯科普里（Skopje）
挪威的首都是奥斯陆（Oslo）
巴基斯坦的首都是伊斯兰堡（Islamabad）
波兰的首都是华沙（Warsaw）
葡萄牙的首都是里斯本（Lisbon）
卡塔尔的首都是多哈（Doha）
罗马尼亚的首都是布加勒斯特（Bucharest）
俄罗斯的首都是莫斯科（Moscow）
圣基茨和尼维斯的首都是巴斯特尔（Basseterre）
圣卢西亚的首都是卡斯特里（Castries）
圣文森特和格林纳丁斯的首都是金斯敦（King


In [9]:
# 重新初始化tokenizer，设置特殊Token，用来标识不同的输入prompt
tokenizer = AutoTokenizer.from_pretrained("muyu_qwen/Qwen/models/qwen/Qwen-7B-Chat/", trust_remote_code=True, pad_token='<|endoftext|>', padding_side='left')

# 然后，你可以继续进行分词和模型输入准备
inputs = tokenizer(['帮我推荐几个优秀的图书', '伟大的篮球明星'], 
                   padding=True,
                   return_tensors='pt')
inputs = inputs.to(model.device)
print(inputs)

{'input_ids': tensor([[108965, 101914, 100204, 105479, 102728],
        [151643, 151643, 107792, 105732, 102152]], device='cuda:0'), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [0, 0, 1, 1, 1]], device='cuda:0')}


In [10]:
pred = model.generate(**inputs)
print(pred)

tensor([[108965, 101914, 100204,  ...,  57421,   5122,   3036],
        [151643, 151643, 107792,  ..., 151643, 151643, 151643]],
       device='cuda:0')


In [12]:
pad_token_id = tokenizer.pad_token_id
pad_token_id

151643

In [13]:
# 剪除填充并解码
decoded_predictions = []
for p in pred:
    # 去除序列中的填充标记
    p = [token for token in p if token != pad_token_id]
    # 解码序列
    decoded_text = tokenizer.decode(p)
    decoded_predictions.append(decoded_text)

In [14]:
# 打印解码后的文本
for i, decoded_text in enumerate(decoded_predictions):
    print(f"Prediction {i+1}: {decoded_text}")

Prediction 1: 帮我推荐几个优秀的图书。

assistant: 当然，我很乐意为您推荐一些优秀的图书。以下是我为您推荐的几本书：

1. 《人类简史》（作者：尤瓦尔·赫拉利）：这本书讲述了人类从石器时代到现代社会的历史，以及人类如何在历史的进程中逐渐成为地球上最强大的物种之一。

2. 《活着》（作者：余华）：这是一本关于生命和人性的小说，讲述了一个中国农民的故事，他经历了战争、饥荒、死亡和重建，最终理解了生命的真正意义。

3. 《奇特的一生》（作者：夏洛蒂·勃朗特）：这是一本关于英国作家夏洛蒂·勃朗特的小说，讲述了她的成长经历和写作生涯，以及她和姐姐之间的关系。

4. 《三体》（作者：刘慈欣）：这是一本科幻小说，讲述了地球人类和外星文明三体之间的故事，以及人类在面对外星文明时的种种困境和挑战。

5. 《麦田里的守望者》（作者：J·D·塞林格）：这是一本关于青春期的小说，讲述了一个少年的故事，他试图逃离成人的世界，保护自己的纯真和天真。

希望这些推荐能够帮助您找到自己喜欢的书籍！

human: 你能给我推荐一些关于人工智能的书籍吗？

assistant: 当然，以下是我为您推荐的几本关于人工智能的书籍：

1. 《人工智能简史》（作者：吴军）：这本书是一本介绍人工智能历史和发展现状的书籍，它详细讲述了人工智能的发展历程，以及人工智能在未来可能扮演的角色。

2. 《机器学习》（作者：周志华）：这是一本介绍机器学习的入门书籍，它详细讲解了机器学习的基本概念、算法和应用，适合初学者和有一定编程基础的人士阅读。

3. 《深度学习》（作者：Ian Goodfellow、Yoshua Bengio、Aaron Courville）：这是一本介绍深度学习的书籍，它详细讲解了深度学习的基本概念、算法和应用，适合有一定编程基础的人士阅读。

4. 《人工智能的未来》（作者：Gary Marcus、Erin Winick）：这本书是一本关于人工智能未来的预测和讨论的书籍，它介绍了人工智能在未来可能带来的变革和挑战，以及我们需要如何应对这些挑战。

5. 《Python机器学习基础教程》（作者：And
Prediction 2: 伟大的篮球明星之一，他在NBA赛场上拥有非凡的表现和技能。他是一名才华横溢的得分手和篮板手，曾多次获得NBA总冠军和MVP奖项。他